In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from typing import Optional, cast, Dict, Any

import torch

import transformers
from transformers import AutoModel, AutoConfig, AutoTokenizer
import datasets

from omegaconf import DictConfig
from omegaconf import OmegaConf as om

from src.flex_bert import *
from src.evals.data import *

/home/public/span/miniconda3/envs/bert24/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/public/span/miniconda3/envs/bert24/lib/python3.11/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/home/public/span/miniconda3/envs/bert24/lib/python3.11/site-packages/flash_attn/ops/triton/layer_norm.py:958: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/home/public/span/miniconda3/envs/bert24/lib/python3.11/site-packages/flash_attn/ops/triton/layer_norm.py:1017: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., 

In [2]:
with open("/home/public/span/MATH_DPO/modern_bert_test/bert24/yamls/main/flex-bert-base-sarah.yaml") as f:
    yaml_config = om.load(f)

cfg = cast(DictConfig, yaml_config)

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained("bclavie/olmo_bert_template")
state_dict = torch.load("/home/public/span/MATH_DPO/modern_bert_test/bert24/checkpoints/latest-rank0.pt")

/home/public/span/temp/ipykernel_885358/594272676.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("/home/public/span/MATH_DPO/modern_bert_test/be

In [4]:
def consume_prefix_in_state_dict_if_present(
    state_dict, prefix
):
    r"""Strip the prefix in state_dict in place, if any.

    ..note::
        Given a `state_dict` from a DP/DDP model, a local model can load it by applying
        `consume_prefix_in_state_dict_if_present(state_dict, "module.")` before calling
        :meth:`torch.nn.Module.load_state_dict`.

    Args:
        state_dict (OrderedDict): a state-dict to be loaded to the model.
        prefix (str): prefix.
    """
    keys = sorted(state_dict.keys())
    for key in keys:
        if key.startswith(prefix):
            newkey = key[len(prefix) :]
            state_dict[newkey] = state_dict.pop(key)

    # also strip the prefix in metadata if any.
    if "_metadata" in state_dict:
        metadata = state_dict["_metadata"]
        for key in list(metadata.keys()):
            # for the metadata dict, the key can be:
            # '': for the DDP module, which we want to remove.
            # 'module': for the actual model.
            # 'module.xx.xx': for the rest.

            if len(key) == 0:
                continue
            newkey = key[len(prefix) :]
            metadata[newkey] = metadata.pop(key)

In [ ]:
state_dict = state_dict['state']['model']
consume_prefix_in_state_dict_if_present(state_dict, "model.")
torch.save(state_dict, "/home/public/span/MATH_DPO/modern_bert_test/bert24/checkpoints/correct_names.pt")

In [43]:
model = create_flex_bert_mlm(
    pretrained_checkpoint=cfg.model.pretrained_checkpoint,
    model_config=cfg.model.model_config,
    tokenizer_name=cfg.tokenizer_name
)
model = model.model.to('cuda')

/home/public/span/MATH_DPO/modern_bert_test/bert24/src/bert_layers/model.py:1063: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(pretrained_checkpoint

In [44]:
ds = datasets.load_dataset("sarahpann/mlm_cls_rewardbench")['train']

In [38]:
ds[8]

{'text': "0[SEP]What are different drawers I should have for clothes?\nThe types of drawers you should have for clothes depend on your personal wardrobe and the space you have available. However, here's a general guide for categorizing different types of clothing into drawers:\n\n1. **Undergarments Drawer**: This drawer is for your underwear, bras, socks, and hosiery. Some people prefer to separate these further, with a drawer specifically for bras and another for underwear and socks.\n\n2. **T-shirts and Casual Tops Drawer**: A drawer for casual wear such as t-shirts, tank tops, and other everyday shirts can help keep your casual wear organized and easily accessible.\n\n3. **Pajamas and Lounge Wear Drawer**: This drawer is for your pajamas, nightgowns, and lounge wear, including comfy shorts and sweatpants.\n\n4. **Activewear and Gym Clothes Drawer**: If you work out regularly, it's helpful to have a drawer dedicated to gym clothes, sports bras, workout t-shirts, leggings, and shorts.

In [9]:
def mask_tokens(ex):
    new_text = "[CLS]" + ex['text'][1:]
    return {"text": new_text, "labels": ex['text']}

In [45]:
masked_in = mask_tokens(ds[8])
print(masked_in)
tokenized = tokenizer(masked_in['text'], return_tensors='pt')
print(tokenized)
input_ids = tokenized['input_ids'].to(device='cuda')
output = model(input_ids)

{'text': "[CLS][SEP]What are different drawers I should have for clothes?\nThe types of drawers you should have for clothes depend on your personal wardrobe and the space you have available. However, here's a general guide for categorizing different types of clothing into drawers:\n\n1. **Undergarments Drawer**: This drawer is for your underwear, bras, socks, and hosiery. Some people prefer to separate these further, with a drawer specifically for bras and another for underwear and socks.\n\n2. **T-shirts and Casual Tops Drawer**: A drawer for casual wear such as t-shirts, tank tops, and other everyday shirts can help keep your casual wear organized and easily accessible.\n\n3. **Pajamas and Lounge Wear Drawer**: This drawer is for your pajamas, nightgowns, and lounge wear, including comfy shorts and sweatpants.\n\n4. **Activewear and Gym Clothes Drawer**: If you work out regularly, it's helpful to have a drawer dedicated to gym clothes, sports bras, workout t-shirts, leggings, and sho

In [46]:
output.logits.shape

torch.Size([1, 644, 50368])

In [41]:
input_ids.shape

torch.Size([1, 644])

In [ ]:
preds = torch.argmax(output.logits, dim=-1)

tensor([[ 5036,  5036,  5036,  2184,  2184,  2824,  2243,  4306,  8384,  3230,
         30714, 49502,  3230,  1426, 47826,  2184, 29409,  2243,  2243,  4306,
         13932, 29616, 47724, 42990,  3230, 34701, 14240, 13932, 29616, 14240,
         18369, 18369, 43716, 14240, 47724, 34701, 27730, 13070, 34701, 13932,
         42990, 42990, 29616,  2243, 42990,   695,  4341, 11427, 11028,  4306,
         30533, 20440,  2243,  4306, 34701, 27730, 40776,  4306, 18369,  4306,
          2243,  4306, 29616,  2243, 29616,  2243,  2243,  4306, 18369, 18369,
          2243, 25551, 18369,  4306,  4306,  4306,  4306, 18369, 31359, 36797,
          4306, 18369, 34701, 47724, 28795, 13932, 34701, 29616, 15852, 47724,
         31921, 45729, 18369,  4306,  4306, 18369, 29616, 18369,  4306, 18369,
         29616,  2243, 29616, 27730, 34701, 29616, 11111, 29616,  4306, 21487,
         45789, 42783,  2243,  7018,  2243,  2243,  2243,  2243,  2243, 15729,
         49673, 18369,  2389, 47945, 14240,  6584, 3

In [48]:
tokenizer.decode(preds[0])

'echechechgogoips momancynextanaoccur Hoeanairc hitchgosci mom momancyuple muminche fermentana Mum circlesuple mum circles neglect neglect velvet circlesinche Mumolerancehelp Mumuple ferment ferment mum mom fermentricocationsviryancy Owumm momancy Mumolerance Hollowancy neglectancy momancy mum mom mum mom momancy neglect neglect mom yoga neglectancyancyancyancy neglect neighbours Mutualancy neglect Mumincheeniumuple Mum mumParentinche fuzzy Hammond neglectancyancy neglect mum neglectancy neglect mum mom mumolerance Mum mum shr mumancy toilet textbooks Eg mom sup mom mom mom mom mom hug Ai neglect love Aviv circlesvioorsancyupleviAgeuple Mumviammers Virgin Virgin mumaways mortliv mat Avivauxplaces dish lawolerance pits mumancy momancy mumancy mum momimeniumweightancy mom mum mom Madancy neglect circles mom mumancyana momancyuruancyury praise mom pulp Gor Mumangles yog deeplyuple momancyancy momolerance Beautyancyancy Mumancyords circles circlesancy mom Specifically mom mumana parenting 